# MrPUP-1: There should be no duplicate (HCPCS, referring NPI, place of service) pair in MrPUP v1.

QA step description: check if there is any duplicate.

Starting Author: Amy Jin (amy@careset.com)

Date: April 30th, 2018

https://docs.google.com/spreadsheets/d/1IYg01IpssJaWHo6KxO4_dSDgXtYNFy41S5cIHFLvlGQ/edit#gid=604789549

## Connect to Parenthood Server

In [1]:
# Packages import
import os
import sys
import numpy as np
import pandas as pd
from collections import Counter
import operator
import mysql.connector
import sshtunnel
import pureyaml

# Handle path
project_dir = !pwd  # dir of current script/notebook file
config_file = open(project_dir[0] + "/db.yaml");
config = pureyaml.load(config_file.read());

# Argument dictionary for sshtunnel
ssh_config = {
    'ssh_address_or_host': ('parenthood.set.care', 22),
    'ssh_username':        config['ssh_username'],
    'ssh_password':        config['ssh_password'],
    'remote_bind_address': ('127.0.0.1', 3306),
    'local_bind_address':  ('0.0.0.0', 3333),
}

# Argument dictionary for mysql.connector
mysql_config = {
    'user':     config['mysql_user'],
    'password': config['mysql_passwd'],
    'host':     config['mysql_host'],
    'database': 'patch',
    'port':     3333,
}

# Connect to Parenthood server
with sshtunnel.SSHTunnelForwarder(**ssh_config) as tunnel:
    print('SSH tunneling successful on port: {}'.format(tunnel.local_bind_port))
    connection = mysql.connector.connect(**mysql_config)
    cur = connection.cursor()
    print('MySQL server connected successfully!')

SSH tunneling successful on port: 3333
MySQL server connected successfully!


## Test Function

In [2]:
# --------------------------------------- Inputs: ---------------------------------------
# 1) db_name:                database name in server
# 2）table_name:             table name
# 3) HCPCS:                  HCPCS code column
# 4) referring_NPI:          referring NPI
# 5) place_of_service:       place of service
# --------------------------------------- Outputs: --------------------------------------
# 1) Test result:            PASS/FAIL
# 2) If FAIL, the test will print out duplicate (HCPCS, referring_NPI, place_of_service).


def mrpup_1(db_name, table_name, HCPCS, referring_NPI, place_of_service):
    #table1 = str(db_name) + '.' + str(table_name)
    with sshtunnel.SSHTunnelForwarder(**ssh_config) as tunnel:
        connection = mysql.connector.connect(**mysql_config)
        cur = connection.cursor()
        print ('Test file: {}.{}'.format(db_name, table_name))
        print ('\n')        
        
        query = ('''
            SELECT *
            FROM {db}.{t1}
            GROUP BY {col1}, {col2}, {col3}
            HAVING COUNT({col1}) >1
            LIMIT 1;
        '''.format(db = db_name, t1 = table_name, col1 = HCPCS, col2 = referring_NPI, col3=place_of_service))

        cur.execute(query)
        rows = list(sum(cur.fetchall(), ()))

        
        if not len(rows):
            print ("Test result: PASS")
        else:
            print ("Test result: FAIL" + '\n')
            print ("The following ({}, {}, {}) pairs have duplicates:".format(HCPCS,referring_NPI,place_of_service) + '\n')
           
            # MySQL query to get dupliate
            query = ('''
                SELECT {col1},{col2}, {col3}, COUNT({col1})
                FROM {db}.{t1}
                GROUP BY {col1}, {col2}, {col3}
                HAVING COUNT({col1}) >1;
            '''.format(db = db_name, t1 = table_name, col1 = HCPCS, col2 = referring_NPI, col3=place_of_service))

            cur.execute(query)
            
            for row in cur.fetchall():
                for i in range(0,len(row)):
                    print (str(row[i]), end=", ")
                print ('\n')
            
        cur.close()
        connection.close()

## Test Example

In [3]:
mrpup_1('_amy', 'test_data_good', 'npi', 'npi_dest', 'npi')

Test file: _amy.test_data_good


Test result: PASS


In [4]:
mrpup_1('_amy', 'test_data_bad1', 'npi', 'npi_dest', 'npi')

Test file: _amy.test_data_bad1


Test result: FAIL

The following (npi, npi_dest, npi) pairs have duplicates:

, , , 8, 

, -4859573525, , 2, 

, 2147483648, , 10, 

, 3599352490, , 2, 

, hello, , 11, 

-1510458706, hello, -1510458706, 2, 

-2387881917, 2147483648, -2387881917, 2, 

-251040449, 2147483648, -251040449, 2, 

-271287451, , -271287451, 2, 

-2916655565, 2147483648, -2916655565, 2, 

-3674108797, hello, -3674108797, 2, 

-6144840502, hello, -6144840502, 2, 

-6321716628, hello, -6321716628, 2, 

-6425644737, hello, -6425644737, 2, 

-6740599147, hello, -6740599147, 2, 

-704006346, 2147483648, -704006346, 2, 

-7086375781, hello, -7086375781, 2, 

-7328876531, hello, -7328876531, 2, 

1003084237, hello, 1003084237, 3, 

1003089814, 2147483648, 1003089814, 2, 

1003090168, 2147483648, 1003090168, 2, 

1003091240, hello, 1003091240, 2, 

1003096272, 2147483648, 1003096272, 2, 

1003096710, hello, 1003096710, 2, 

1003096785, hello, 1003096785, 2, 

1003099284, hello, 1003099